# Video pipeline with Yolo + Resnet Model

In [1]:
import os
import sys
from pprint import PrettyPrinter
import matplotlib.pyplot as plt
from typing import Dict, List
import numpy as np

pp = PrettyPrinter(indent=4)
from barazmoon.twitter import twitter_workload_generator

# get an absolute path to the directory that contains parent files
__file__ = globals()["_dh"][0]
project_dir = __file__ = globals()["_dh"][0]
sys.path.append(os.path.normpath(os.path.join(project_dir, "..", "..", "..")))

from experiments.utils.constants import FINAL_RESULTS_PATH
from experiments.utils.parser import AdaptationParser
from experiments.utils.drawing import draw_temporal, draw_cumulative

In [2]:
metaserieses = [23, 23, 23, 23, 23]
serieses = [1, 21, 41, 61, 81]

series_names = None
# series_names = {
#     # 1: "IPA-accuracy",
#     2: "IPA",
#     # 3: "IPA-cost",
#     4: "FA2-high",
#     # 5: "FA2-high",
#     # 6: "RIM-low",
#     7: "RIM-high",
# }
factors = {1: 2 / 3, 21: 2, 41: 2, 61: 1, 81: 2 / 3}
series_paths = {
    series: os.path.join(
        FINAL_RESULTS_PATH, "metaseries", str(metaseries), "series", str(series)
    )
    for series, metaseries in zip(serieses, metaserieses)
}

loaders = {
    series: AdaptationParser(
        series_path=series_path, model_name="video", type_of="router_pipeline"
    )
    for series, series_path in series_paths.items()
}

In [3]:
simulation_modes = {}
configs = {}
threshold = {}
for series, loader in loaders.items():
    configs_exp = loader.load_configs()
    config = configs_exp["0.yaml"]
    configs[series] = config
    simulation_modes[series] = config["simulation_mode"]
    threshold[series] = config["threshold"]
threshold

{1: 4, 21: 1, 41: 1, 61: 5, 81: 4}

In [4]:
# load the sent workload
sent_loads = {}
for series, config in configs.items():
    workload_type = config["workload_type"]
    workload_config = config["workload_config"][0]
    start = workload_config["start"]
    end = workload_config["end"]
    damping_factor = workload_config["damping_factor"]
    sent_loads[series] = twitter_workload_generator(
        days=f"{start}-{end}", damping_factor=damping_factor
    )

In [5]:
adaptation_logs = dict(
    map(lambda l: (l[0], l[1].load_adaptation_log()), loaders.items())
)

In [6]:
base_allocations = {}
# stage_wise_slas = {}
series_changes = {}
for series in serieses:
    series_changes[series] = loaders[series].series_changes(
        adaptation_log=adaptation_logs[series]
    )
    base_allocations[series] = series_changes[series]["base_allocations"]
pp.pprint(base_allocations)

{   1: {   'classification': {   'resnet101': 1,
                                 'resnet152': 2,
                                 'resnet18': 1,
                                 'resnet34': 1,
                                 'resnet50': 1},
           'crop': {   'yolov5l': 4,
                       'yolov5m': 2,
                       'yolov5n': 1,
                       'yolov5s': 1,
                       'yolov5x': 8}},
    21: {   'audio': {   'facebook-s2t-large-librispeech-asr': 4,
                         'facebook-s2t-medium-librispeech-asr': 2,
                         'facebook-s2t-small-librispeech-asr': 1,
                         'facebook-wav2vec2-base-960h': 2,
                         'facebook-wav2vec2-large-960h': 8},
            'nlp-qa': {   'deepset-roberta-base-squad2': 1,
                          'deepset-xlm-roberta-base-squad2': 1}},
    41: {   'audio': {   'facebook-s2t-large-librispeech-asr': 8,
                         'facebook-s2t-medium-librispeech-a